In [1]:
from datetime import datetime, timedelta
from strategy.stratfact import StratFact
from backtester.backtester import Backtester
from portfolio.equity_portfolio import EquityPortfolio
from processor.processor import Processor as p
from tqdm import tqdm
import pytz
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
start = datetime(2016,1,1)
end = datetime(2022,1,1)
seats = 10

In [3]:
# port = EquityPortfolio(start,end,"portfolio",strats = {"rolling_percent":{"modeling_params":{"days":100,"currency":"prices"},
#                         "trading_params":{"requirement":5,"value":True}},
# "progress_report":{"modeling_params":{}
#                    ,"trading_params":{"value":True,"requirement":5}},
# "financial_predict":{"modeling_params":{"score_requirement":70,"categories":4,"model_training_year":4},
#                     "trading_params":{"requirement":10,"value":True}},
# "speculation":{"modeling_params":{"number_of_training_weeks":14,"categories":4
#                                   ,"model_training_year":1,"score_requirement":70},
#                "trading_params":{"score_requirement":70,"requirement":5,"value":True}},
# "competition":{"modeling_params":{"categories":4,"model_training_year":1,"score_requirement":70},
#                 "trading_params":{"score_requirement":70,"requirement":5,"value":True}}})

In [3]:
port = EquityPortfolio(start,end,"industry_price_portfolio",strats = 
{"rolling_percent":{"modeling_params":{"days":100,"currency":"prices"},
                        "trading_params":{"requirement":5,"value":True}},
"progress_report":{"modeling_params":{}
                   ,"trading_params":{"value":True,"requirement":5}},
"financial_predict":{"modeling_params":{"score_requirement":70,"categories":3,"model_training_year":4},
                    "trading_params":{"requirement":10,"value":True}},
"speculation":{"modeling_params":{"number_of_training_weeks":14,"categories":3
                                  ,"model_training_year":4,"score_requirement":70},
               "trading_params":{"score_requirement":70,"requirement":5,"value":True}},
"competition":{"modeling_params":{"categories":3,"model_training_year":4,"score_requirement":70},
                "trading_params":{"score_requirement":70,"requirement":5,"value":True}}})

In [4]:
port.load()

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:23<00:00,  4.72s/it]


In [5]:
port.transform()

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:16<00:00,  3.21s/it]


In [ ]:
port.

In [8]:
port.strats["speculation"]["class"].db.connect()
port.strats["speculation"]["class"].db.delete("sim",{"categories":3})
port.strats["speculation"]["class"].db.disconnect()

In [ ]:
port.sim()

speculation_sim_year:   0%|                                                                                | 0/7 [00:00<?, ?it/s]

200Communication Services list index out of range


In [ ]:
port.strats["rolling_percent"]

In [ ]:
trades = port.backtest()

In [ ]:
at = trades
strats = trades["strategy"].unique()
all_analysis = []
for strat in strats:
    strat_trades = at[at["strategy"]==strat]
    cash = []
    stuff = []
    total_cash = 100
    number_of_strats=1
    for seat in range(seats):
        initial = float(total_cash / number_of_strats / seats )
        seat_trades = strat_trades[strat_trades["seat"]==seat]
        for delta in seat_trades["delta"]:
            initial = initial * (1+delta)
            cash.append(initial)
    strat_trades["pv"] = cash
    stuff.append(strat_trades)
    analysis = pd.concat(stuff).pivot_table(index=["sell_date"],columns="seat",values="pv").fillna(method="ffill").fillna(float(total_cash / number_of_strats / seats )).reset_index()
    analysis["pv"] = [sum([row[1][i] for i in range(seats)]) for row in analysis.iterrows()]
    analysis["strategy"] = strat
    all_analysis.append(analysis)
    plt.plot(analysis["sell_date"],analysis["pv"])
plt.legend(strats)
plt.show()

In [ ]:
hl = pd.concat(all_analysis)
hl

In [ ]:
hl.pivot_table(index="sell_date",columns="strategy",values="pv").fillna(method="ffill").fillna(100)